In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data102217


In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
train=pd.read_csv('/home/aistudio/data/data102217/train.csv')
test=pd.read_csv('/home/aistudio/data/data102217/test1.csv')
test=test.iloc[:,1:]
train=train.iloc[:,1:]

In [5]:
features=train.columns.tolist()
features.remove('label')
print(features)

['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi', 'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'sid', 'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type']


In [6]:
for feature in features:
    
     print(feature,train[feature].nunique())

android_id 362258
apptype 89
carrier 5
dev_height 798
dev_ppi 92
dev_width 346
lan 21
media_id 284
ntt 8
os 2
osv 154
package 1950
sid 500000
timestamp 500000
version 22
fea_hash 402980
location 332
fea1_hash 4959
cus_type 58


In [7]:
#对osv进行数据清洗
def osv_trans(x):
    x=str(x).replace('Android_','').replace('Android ','').replace('W','')
    if x.find('.')>0:
        temp_index1=x.find('.')
        if x.find(' ')>0:
            temp_index2=x.find(' ')
        else:
            temp_index2=len(x)    
        if x.find('-')>0:
            temp_index2=x.find('-')
       
        result=x[0:temp_index1]+'.'+x[temp_index1+1:temp_index2].replace('.','')
    try:
        return float(result)
    except:
        return 0
    try:
        return float(x)
    except:
        print(x)
        return 0
#采用众数进行缺失值的填充
train['osv'].fillna('8.1.0',inplace=True)
#数据清洗
train['osv']=train['osv'].apply(osv_trans)

#采用众数进行缺失值的填充
test['osv'].fillna('8.1.0',inplace=True)
#数据清洗
test['osv']=test['osv'].apply(osv_trans)

In [8]:
#train['os'].value_counts()
train['lan'].value_counts()
#lan_map = {'zh-CN': 1, }
train['lan'].value_counts().index
lan_map = {'zh-CN': 1, 'zh_CN':2, 'Zh-CN': 3, 'zh-cn': 4, 'zh_CN_#Hans':5, 'zh': 6, 'ZH': 7, 'cn':8, 'CN':9, 'zh-HK': 10, 'tw': 11, 'TW': 12, 'zh-TW': 13,             'zh-MO':14, 'en':15, 'en-GB': 16, 'en-US': 17, 'ko': 18, 'ja': 19, 'it': 20, 'mi':21} 
train['lan'] = train['lan'].map(lan_map)
test['lan'] = test['lan'].map(lan_map)
test['lan'].value_counts()


# In[6]:


# 对于有缺失的lan 设置为19
train['lan'].fillna(19, inplace=True)
test['lan'].fillna(19, inplace=True)

In [9]:
remove_list=['os', 'sid']
col=features
for i in remove_list:
    col.remove(i)
col

['android_id',
 'apptype',
 'carrier',
 'dev_height',
 'dev_ppi',
 'dev_width',
 'lan',
 'media_id',
 'ntt',
 'osv',
 'package',
 'timestamp',
 'version',
 'fea_hash',
 'location',
 'fea1_hash',
 'cus_type']

In [10]:
def version_trans(x):
    if x=='V3':
        return 3
    if x=='v1':
        return 1
    if x=='P_Final_6':
        return 6
    if x=='V6':
        return 6
    if x=='GA3':
        return 3
    if x=='GA2':
        return 2
    if x=='V2':
        return 2
    if x=='50':
        return 5
    return int(x)
train['version'] = train['version'].apply(version_trans)
test['version'] = test['version'].apply(version_trans)
train['version'] = train['version'].astype('int')
test['version'] = test['version'].astype('int')

In [11]:
#特征筛选
features=train[col]
# 构造fea_hash_len特征
features['fea_hash_len'] = features['fea_hash'].map(lambda x: len(str(x)))
features['fea1_hash_len'] = features['fea1_hash'].map(lambda x: len(str(x)))
# Thinking：为什么将很大的，很长的fea_hash化为0？
# 如果fea_hash很长，都归为0，否则为自己的本身
features['fea_hash'] = features['fea_hash'].map(lambda x: 0 if len(str(x))>10 else int(x))
features['fea1_hash'] = features['fea1_hash'].map(lambda x: 0 if len(str(x))>10 else int(x))
features

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,fea_hash_len,fea1_hash_len
0,316361,1199,46000.0,0.0,0.0,0.0,19.0,104,6.0,0.00,18,1.559893e+12,8,2135019403,0,2329670524,601,10,10
1,135939,893,0.0,0.0,0.0,0.0,19.0,19,6.0,8.10,0,1.559994e+12,4,2782306428,1,2864801071,1000,10,10
2,399254,821,0.0,760.0,0.0,360.0,19.0,559,0.0,8.10,0,1.559837e+12,0,1392806005,2,628911675,696,10,9
3,68983,1004,46000.0,2214.0,0.0,1080.0,19.0,129,2.0,8.10,0,1.560042e+12,0,3562553457,3,1283809327,753,10,10
4,288999,1076,46000.0,2280.0,0.0,1080.0,1.0,64,2.0,8.00,0,1.559867e+12,5,2364522023,4,1510695983,582,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,392477,1028,46000.0,1920.0,3.0,1080.0,1.0,144,6.0,7.12,25,1.559834e+12,7,861755946,79,140647032,373,9,9
499996,346134,1001,0.0,1424.0,0.0,720.0,19.0,29,2.0,8.10,0,1.559814e+12,3,1714444511,23,2745131047,525,10,10
499997,499635,761,46000.0,1280.0,0.0,720.0,19.0,54,6.0,6.01,9,1.559676e+12,0,3843262581,25,1326115882,810,10,10
499998,239786,917,46001.0,960.0,0.0,540.0,2.0,109,2.0,5.11,0,1.559840e+12,0,1984296118,225,1446741112,772,10,10


In [12]:
test_features = test[col]
# 构造fea_hash_len特征
test_features['fea_hash_len'] = test_features['fea_hash'].map(lambda x: len(str(x)))
test_features['fea1_hash_len'] = test_features['fea1_hash'].map(lambda x: len(str(x)))
# Thinking：为什么将很大的，很长的fea_hash化为0？
# 如果fea_hash很长，都归为0，否则为自己的本身
test_features['fea_hash'] = test_features['fea_hash'].map(lambda x: 0 if len(str(x))>10 else int(x))
test_features['fea1_hash'] = test_features['fea1_hash'].map(lambda x: 0 if len(str(x))>10 else int(x))
test_features

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,fea_hash_len,fea1_hash_len
0,317625,1181,46000.0,2196.0,2.0,1080.0,9.0,639,2.0,8.10,188,1.559872e+12,7,1672223856,57,3872258917,658,10,10
1,435108,944,46003.0,2280.0,3.0,1080.0,1.0,704,6.0,8.10,221,1.559739e+12,3,3767901757,23,129322164,943,10,9
2,0,1106,46000.0,0.0,0.0,0.0,19.0,39,2.0,5.10,1562,1.559614e+12,0,454638703,30,4226678391,411,9,10
3,451504,761,46000.0,1344.0,0.0,720.0,19.0,54,2.0,7.11,9,1.559668e+12,0,1507622951,65,3355419572,848,10,10
4,0,1001,46000.0,665.0,0.0,320.0,1.0,29,5.0,8.10,4,1.559694e+12,0,4116351093,148,2644467751,411,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0,1001,46000.0,760.0,0.0,360.0,1.0,29,2.0,8.10,4,1.559957e+12,0,3162887451,126,2711576615,411,10,10
149996,0,1001,46000.0,780.0,0.0,360.0,1.0,29,2.0,9.00,4,1.559863e+12,0,97238959,322,2678022183,411,8,10
149997,0,1001,46000.0,780.0,0.0,360.0,1.0,29,5.0,8.10,4,1.560041e+12,0,1320118495,46,2610913319,411,10,10
149998,500925,1052,46000.0,854.0,240.0,480.0,1.0,249,6.0,4.42,0,1.559688e+12,2,1292986591,41,1898209327,430,10,10


In [13]:
#test['version'].value_counts()
#features['version'].value_counts()
features['dev_height'].value_counts()
features['dev_width'].value_counts()
# 构造面积特征
features['dev_area'] = features['dev_height'] * features['dev_width']
test_features['dev_area'] = test_features['dev_height'] * test_features['dev_width']


In [14]:
"""
Thinking：是否可以利用 dev_ppi 和 dev_area构造新特征
features['dev_ppi'].value_counts()
features['dev_area'].astype('float') / features['dev_ppi'].astype('float')
"""
#features['ntt'].value_counts()
features['carrier'].value_counts()
features['package'].value_counts()
# version - osv APP版本与操作系统版本差
features['osv'].value_counts()
features['version_osv'] = features['osv'] - features['version']
test_features['version_osv'] = test_features['osv'] - test_features['version']

In [15]:
#从train里找到selected字段的，关键特征值
def find_key_feature(train,selected):
    temp0=train[train['label']==0]
    temp=pd.DataFrame(columns=[0,1])
    temp[0]=temp0['osv'].value_counts()/len(temp0)*100

    temp1=train[train['label']==1]
    temp[1]=temp1['osv'].value_counts()/len(temp1)*100
    temp[2]=temp[1]/temp[0]
    #筛选出特征比大于10的特征
    result=temp[temp[2]>3].sort_values(2,ascending=False).index
    return result

key_feature={}
key_feature['osv']=find_key_feature(train,'osv')
key_feature

{'osv': Float64Index([7.77, 7.21, 7.75, 7.85, 7.87,  3.8, 7.67,  3.9,  2.3, 8.01,  7.9,
               7.64, 3.84, 7.89, 7.92,  4.1, 7.72, 7.82,  7.8, 3.86,  7.7, 7.84,
               7.68, 7.86, 7.73,  9.0,  6.1, 3.83, 3.78, 7.74, 4.03, 6.12, 9.05,
               4.12,  0.0],
              dtype='float64')}

In [16]:
#重要特征为1 否则为0
def f(x):
    if x in key_feature['osv']:
        return 1
    else:
        return 0
features['osv1']=features['osv'].apply(f)
test_features['osv1']=test_features['osv'].apply(f)

In [26]:

#train['os'].value_counts()
# 使用XGBoost训练

import xgboost as xgb
model_xgb= xgb.XGBClassifier(
            max_depth=15, learning_rate=0.05, n_estimators=5000, 
            objective='binary:logistic',tree_method='gpu_hist', 
            subsample=0.8, colsample_bytree=0.8, 
            eval_metric='auc', reg_lambda=0.5
        )


# 模型训练
model_xgb.fit(features.drop(['timestamp'], axis=1), train['label'])
result_xgb = model_xgb.predict(test_features.drop(['timestamp'], axis=1))
result_xgb

array([0, 1, 0, ..., 1, 1, 1])

In [27]:
#features['version'].value_counts()
res = pd.DataFrame(test['sid'])
res['label'] = result_xgb
res.to_csv('./xgb-15-0.8.csv', index=False)
res

,sid,label
0,1440682,0
1,1606824,1
2,1774642,0
3,1742535,0
4,1689686,1
...,...,...
149995,1165373,1
149996,1444115,1
149997,1134378,1
149998,1700238,1


In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 